## Initialization

In [1]:
struct Siddon
    origin::Vector{Float64}
    target::Vector{Float64}
    spacing::Vector{Float64}
    isocenter::Vector{Float64}
    dims::Vector{Int64}
end

In [2]:
function get_α(i::Int64, j::Int64; sid::Siddon)
    planes = [i, j]
    return @. (sid.isocenter + planes * sid.spacing - sid.origin) / (sid.target - sid.origin)
end


function get_α_minmax(sid::Siddon)    
    αx0, αy0 = get_α(0, 0; sid)
    αx1, αy1 = sid.dims .- 1 |> nxy -> get_α(nxy...; sid)
    αxmin, αxmax = minmax(αx0, αx1)
    αymin, αymax = minmax(αy0, αy1)
    αmin = max(αxmin, αymin)
    αmax = min(αxmax, αymax)
    return αxmin, αxmax, αymin, αymax, αmin, αmax
end

get_α_minmax (generic function with 1 method)

In [3]:
function get_φ(α::Float64; sid::Siddon)
    pxyz = @. sid.origin + α * (target - origin)
    return @. (pxyz - sid.isocenter) / sid.spacing
end

get_φ (generic function with 1 method)

In [4]:
function get_idx_minmax(αmin, αmax, αxmin, αxmax, ixmin, ixmax, p1, p2)
    if p1 ≤ p2
        imin = αmin == αxmin ? 1 : Int(ceil(ixmin))
        imax = αmax == αxmax ? nx-1 : Int(floor(ixmax))
    else
        imin = αmax == αxmax ? 1 : Int(ceil(ixmax))
        imax = αmin == αxmin ? nx-2 : Int(floor(ixmin))
    end
    return imin, imax
end


function initialize(sid::Siddon)
    αxmin, αxmax, αymin, αymax, αmin, αmax = get_α_minmax(sid)
    ixmin, jxmin = get_φ(αmin; sid)
    ixmax, jxmax = get_φ(αmax; sid)
    imin, imax = get_idx_minmax(αmin, αmax, αxmin, αxmax, ixmin, ixmax, sid.origin[1], sid.target[1])
    jmin, jmax = get_idx_minmax(αmin, αmax, αymin, αymax, jxmin, jxmax, sid.origin[2], sid.target[2])
    return αmin, αmax, imin, imax, jmin, jmax
end

initialize (generic function with 1 method)

In [5]:
# Little test implementation
ΔX, ΔY = 1, 1
X₀, Y₀ = 0, 0

nx = 5  # Number of planes (x)
ny = 5  # Number of planes (y)

origin = [-0.5, 0.5]
target = [4.5, 4.5]
spacing = [ΔX, ΔY]
isocenter = [X₀, Y₀]
dims = [nx, ny]

sid = Siddon(origin, target, spacing, isocenter, dims)

Siddon([-0.5, 0.5], [4.5, 4.5], [1.0, 1.0], [0.0, 0.0], [5, 5])

In [6]:
# Test the initialization function
αmin, αmax, imin, _, jmin, _ = initialize(sid)

(0.1, 0.875, 1, 3, 1, 4)

## Loop

In [7]:
CartesianIndex(trunc.(Int, get_φ(0.1; sid))...)

CartesianIndex(0, 0)

In [8]:
get_vox_idx(α::Float64) = trunc.(Int, get_φ(α; sid)) .+ 1

get_vox_idx (generic function with 1 method)

In [9]:
iu = sid.origin[1] ≤ sid.target[1] ? 1 : -1
ju = sid.origin[2] ≤ sid.target[2] ? 1 : -1

update_i = [iu, ju]
update_α = @. sid.spacing / abs(sid.target-sid.origin)

αcurr = αmin

αs = get_α(imin, jmin; sid)
αnext, idx = findmin(αs)

αmid = (αnext + αcurr) / 2
voxs = get_vox_idx(αmid)

2-element Vector{Int64}:
 1
 1

In [10]:
len = αnext - αcurr
@show αnext, len, voxs

while αcurr < αmax
    voxs[idx] += update_i[idx]
    αs[idx] += update_α[idx]
    
    αnext, idx = findmin(αs)
    
    len = αnext - αcurr
    @show αnext, len, voxs
    
    αcurr = αnext
end

(αnext, len, voxs) = (0.125, 0.024999999999999994, [1, 1])
(αnext, len, voxs) = (0.3, 0.19999999999999998, [1, 2])
(αnext, len, voxs) = (0.375, 0.07500000000000001, [2, 2])
(αnext, len, voxs) = (0.5, 0.125, [2, 3])
(αnext, len, voxs) = (0.625, 0.125, [3, 3])
(αnext, len, voxs) = (0.7, 0.07499999999999996, [3, 4])
(αnext, len, voxs) = (0.875, 0.17500000000000004, [4, 4])
